In [ ]:
import matplotlib.pyplot as pp
import numpy as np

In [ ]:
pp.style.use('ggplot')

In [ ]:
def kernel(x, y, length=1.0, sigma=1.0):
    distance_squared = np.sum(x**2, 1).reshape(-1, 1) + np.sum(y**2, 1) - 2 * np.dot(x, y.T)
    return sigma**2 * np.exp(-distance_squared / 2 / length**2)

x = np.arange(-5, 5, 0.1).reshape(-1, 1)
mu = np.zeros(x.shape)
cov = kernel(x, x)
y = np.random.multivariate_normal(mu.ravel(), cov, 3).T

pp.plot(x, y)

In [ ]:
def posterior_predictive(x_s, x, y, length_f=1.0, sigma_f=1.0, sigma_y=0):
    k = kernel(x, x, length_f, sigma_f)
    k_s = kernel(x_s, x, length_f, sigma_f)
    k_ss = kernel(x_s, x_s, length_f, sigma_f)
    k_inv = np.linalg.inv(k + sigma_y**2 * np.eye(len(x)))
    mu_s = k_s.dot(k_inv).dot(y)
    cov_s = k_ss - k_s.dot(k_inv).dot(k_s.T)
    return mu_s, cov_s

In [ ]:
x_train = np.array([-4, -3, -2, -1, 1]).reshape(-1, 1)
y_train = np.sin(x_train)

mu, cov = posterior_predictive(x, x_train, y_train)
y = np.random.multivariate_normal(mu.ravel(), cov, 3).T

pp.plot(x, y)
pp.scatter(x_train, y_train)